## Neural networks project

This is a datascience training project in which I had to create a neural network to predict the kind of trees will grow in an area based on cartographic data

### 1st I loaded the necessary libraries

In [1]:
import pandas as pd
import numpy as np
import os

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import  Dense, Dropout


from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

### Then I loaded and inspected the data, as usual

In [2]:
data = pd.read_csv("cover_data.csv")

In [3]:
data.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,class
0,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,0,0,0,0,5
2,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,0,0,0,0,2
3,2785,155,18,242,118,3090,238,238,122,6211,...,0,0,0,0,0,0,0,0,0,2
4,2595,45,2,153,-1,391,220,234,150,6172,...,0,0,0,0,0,0,0,0,0,5


To avoid writing all the features one-by-one, this command gives me a list of columns, I'll just need to remove the last one that gives the tree-cover type, which is what we are training the algorithm to predict:

In [4]:
data.columns

Index(['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
       'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
       'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
       'Horizontal_Distance_To_Fire_Points', 'Wilderness_Area1',
       'Wilderness_Area2', 'Wilderness_Area3', 'Wilderness_Area4',
       'Soil_Type1', 'Soil_Type2', 'Soil_Type3', 'Soil_Type4', 'Soil_Type5',
       'Soil_Type6', 'Soil_Type7', 'Soil_Type8', 'Soil_Type9', 'Soil_Type10',
       'Soil_Type11', 'Soil_Type12', 'Soil_Type13', 'Soil_Type14',
       'Soil_Type15', 'Soil_Type16', 'Soil_Type17', 'Soil_Type18',
       'Soil_Type19', 'Soil_Type20', 'Soil_Type21', 'Soil_Type22',
       'Soil_Type23', 'Soil_Type24', 'Soil_Type25', 'Soil_Type26',
       'Soil_Type27', 'Soil_Type28', 'Soil_Type29', 'Soil_Type30',
       'Soil_Type31', 'Soil_Type32', 'Soil_Type33', 'Soil_Type34',
       'Soil_Type35', 'Soil_Type36', 'Soil_Type37', 'Soil_Type38',
       'Soil_Type39', 'Soil_Type40

This command gets me a list of all the cartographic data into a dataframe, this is going to be our input in the neural network:

In [5]:
features = data[['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
       'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
       'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
       'Horizontal_Distance_To_Fire_Points', 'Wilderness_Area1',
       'Wilderness_Area2', 'Wilderness_Area3', 'Wilderness_Area4',
       'Soil_Type1', 'Soil_Type2', 'Soil_Type3', 'Soil_Type4', 'Soil_Type5',
       'Soil_Type6', 'Soil_Type7', 'Soil_Type8', 'Soil_Type9', 'Soil_Type10',
       'Soil_Type11', 'Soil_Type12', 'Soil_Type13', 'Soil_Type14',
       'Soil_Type15', 'Soil_Type16', 'Soil_Type17', 'Soil_Type18',
       'Soil_Type19', 'Soil_Type20', 'Soil_Type21', 'Soil_Type22',
       'Soil_Type23', 'Soil_Type24', 'Soil_Type25', 'Soil_Type26',
       'Soil_Type27', 'Soil_Type28', 'Soil_Type29', 'Soil_Type30',
       'Soil_Type31', 'Soil_Type32', 'Soil_Type33', 'Soil_Type34',
       'Soil_Type35', 'Soil_Type36', 'Soil_Type37', 'Soil_Type38',
       'Soil_Type39', 'Soil_Type40']]

I also need to create a dataframe with the labels, which is what we want to predict with the algorythm:

In [6]:
labels = data["class"]

Looking at the shape of the labels, it seems that type 1 and type 2 treecovers are the most abundant:

In [7]:
labels.value_counts()

2    283301
1    211840
3     35754
7     20510
6     17367
5      9493
4      2747
Name: class, dtype: int64

In most machine learning models, including neural networks, we usually start by dividing the data into a training set and a testing set. 

Like the name indicates, the training set is used to train the neural network, and the test set is used to evaluate it's accuracy:

In [8]:
x_train, x_test, y_train, y_test = train_test_split(features, labels)

Before we start, we need to make sure that the data is normalized, we don't want any input to have disproportionate weight. We'll normalize it in a way that it's average becomes 0 and std deviation becomes 1:

In [9]:
ct = StandardScaler()
x_train = ct.fit_transform(x_train)
x_test = ct.transform(x_test)

Now we are ready to create the neural network, we start by chosing which kind of model:

In [10]:
model = Sequential()

We create the imput layer, which corresponds to the number of features:

In [11]:
model.add(tf.keras.layers.InputLayer(input_shape=(54,)))

Then we build a brain, this is the customizable step, and where the name deep learning comes from, we can decide how deep the brain is, and how many neurons it contains, balancing the accuracy of the model and the time it takes to compute.

After some configuration attempts, this is the brain architecture I came up with: 

In [12]:
model.add(Dense(356, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(8, activation='softmax'))

We create a script to train the neural network, with the parameters we are optimizing:

In [13]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

And we input our training data, with 100 epochs the accuracy of the model is 93%. You can reduced the number of epochs if you are in a hurry, althought the accuracy won't be as good:

In [14]:
model.fit(x_train, y_train, epochs = 100, batch_size = 5000)

Epoch 1/100
88/88 [==============================] - 7s 42ms/step - loss: 0.8147 - sparse_categorical_accuracy: 0.6862
Epoch 2/100
88/88 [==============================] - 4s 42ms/step - loss: 0.5746 - sparse_categorical_accuracy: 0.7563
Epoch 3/100
88/88 [==============================] - 4s 44ms/step - loss: 0.5217 - sparse_categorical_accuracy: 0.7778
Epoch 4/100
88/88 [==============================] - 4s 46ms/step - loss: 0.4842 - sparse_categorical_accuracy: 0.7947
Epoch 5/100
88/88 [==============================] - 5s 54ms/step - loss: 0.4550 - sparse_categorical_accuracy: 0.8074
Epoch 6/100
88/88 [==============================] - 5s 55ms/step - loss: 0.4339 - sparse_categorical_accuracy: 0.8169
Epoch 7/100
88/88 [==============================] - 5s 53ms/step - loss: 0.4116 - sparse_categorical_accuracy: 0.8279
Epoch 8/100
88/88 [==============================] - 5s 52ms/step - loss: 0.3962 - sparse_categorical_accuracy: 0.8353
Epoch 9/100
88/88 [=============================

88/88 [==============================] - 4s 42ms/step - loss: 0.1942 - sparse_categorical_accuracy: 0.9228
Epoch 70/100
88/88 [==============================] - 4s 41ms/step - loss: 0.1922 - sparse_categorical_accuracy: 0.9234
Epoch 71/100
88/88 [==============================] - 4s 43ms/step - loss: 0.1877 - sparse_categorical_accuracy: 0.9259
Epoch 72/100
88/88 [==============================] - 4s 41ms/step - loss: 0.1870 - sparse_categorical_accuracy: 0.9259
Epoch 73/100
88/88 [==============================] - 4s 41ms/step - loss: 0.1881 - sparse_categorical_accuracy: 0.9249
Epoch 74/100
88/88 [==============================] - 4s 41ms/step - loss: 0.1860 - sparse_categorical_accuracy: 0.9262
Epoch 75/100
88/88 [==============================] - 4s 41ms/step - loss: 0.1848 - sparse_categorical_accuracy: 0.9266
Epoch 76/100
88/88 [==============================] - 4s 41ms/step - loss: 0.1857 - sparse_categorical_accuracy: 0.9265
Epoch 77/100
88/88 [==============================] -

And we test the model on our validation data, getting a final report about the performance of the model:

In [15]:
y_estimate = model.predict(x_test)
y_estimate = np.argmax(y_estimate, axis = 1)
print(classification_report(y_test, y_estimate))

4540/4540 [==============================] - 11s 2ms/step
              precision    recall  f1-score   support

           1       0.94      0.90      0.92     52661
           2       0.93      0.95      0.94     71024
           3       0.91      0.93      0.92      8851
           4       0.88      0.77      0.82       702
           5       0.81      0.81      0.81      2417
           6       0.85      0.83      0.84      4389
           7       0.92      0.95      0.93      5209

    accuracy                           0.93    145253
   macro avg       0.89      0.88      0.88    145253
weighted avg       0.93      0.93      0.93    145253



Final comment: neural networks have the advantage of being very efficient at classifing data, but they have the problem of being "blackboxes", in which it is almost impossible to get insight on what is the weight of each input to the final prediction.

Usually, when trying to understand the why is more important than the final classification, using other machine learning methods is more appropriate.